In [3]:
# imports(default)
import sys
from glob import glob

In [4]:
charactor_files = glob("../dataset/charactor/*.parsed")
conversation_files = glob("../dataset/conversation/nucc_*.parsed")

In [5]:
all_files = charactor_files + conversation_files

In [6]:
def read_file(file):
    with open(file, "r") as f:
        res = f.readlines()
    return res
all_texts = [read_file(file) for file in all_files]

In [8]:
all_texts[0]

['▁こ 、 こないだ は 全く 歯 が 立 た た なかった ぞ ぉ ! あ ... あんな こと された の っ、 初めて にゃ ... 、 ちゃ ぁ 〜 んと 責 任 、 とって よね っ ☆ さ ぁ 、 みく を ト ップ アイドル に 仕 立て 上げ るの にゃ ☆\n',
 '▁ふ にゃ ぁ 〜 ... ね む ね む ...\n',
 '▁お っ は にゃ〜 っ ☆\n',
 '▁ 八 重 歯 、 カワイ イ っ しょ 〜 ☆\n',
 '▁ん ん 〜 っ! ? みく に 見 と れてる のか にゃ Pチャン ? * 1\n',
 '▁お ー しま いっ !\n',
 '▁お 仕事 は マジ メ なの っ!\n',
 '▁ま っか せ っ にゃ さ ぁ 〜 い ! !\n',
 '▁どう よ Pチャン ! 今日 も みくは 可 愛 かった でしょ 〜 ☆\n',
 '▁Pチャン 知ってる? みくは 簡単 にな つ かない の にゃ〜\n',
 '▁い っく にゃ ー ! !\n',
 '▁Pチャン っていい ひ と だった んだ ね 〜 。 き ま ぐ れ な みく でも、 これは ちょっと ついて い きた くな っちゃう かも にゃ〜 ♪\n',
 '▁ ( Pチャン 発 見 ! よ っしゃ 、 跳 び かか って やる にゃ ... )\n',
 '▁ にゃ は は っ! Pチャン の ために がん ば った にゃ!\n',
 '▁う っ にゃ〜 ! ねこ みみ に、 ねこ し っぽ っ、 カ ッ ワイ イ 衣 装 だ ぁ 〜 っ ☆ こんなに みく のこと 喜 ば せて くれ ちゃ って 〜 。 まった く もう、 プロ デ ュー サー チャン は ナ ニ 考え てる んだろう にゃ ぁ ♪ * 2\n',
 '▁ふ にゃ ぁ 〜 ... ね む ね む ...\n',
 '▁お っ は にゃ〜 っ ☆\n',
 '▁Pチャン ... えい っ、 ねこ ぱ んち ☆ 特に 意味 はない にゃ\n',
 '▁お 魚 、 実は 苦 手 ...... 秘 密 だ ぞ !\n',
 '▁ま っか せ っ にゃ さ ぁ 〜 い ! !\n',
 '▁お ー しま いっ !\n',
 '▁ にゃん にゃん ダン ス って どう ?\n',
 '▁じゃ ね 